In [2]:
from google.colab import drive
import os
import sys
import pandas as pd
import random
from math import exp, log
import pickle as pkl

class Point:
    def __str__(self):
        return "<{}:{}>".format(self.label, self.values)
    def __repr__(self):
        return "<{}:{}>".format(self.label, self.values)
    def __init__(self, label, values):
        self.label = label
        self.values = values

def load_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    points = []
    for _, row in data.iterrows():
        label = row['Class']
        vals = row.drop(['Time', 'Class']).values.tolist()
        points.append(Point(label, vals))
    return points

def logistic(x):
    return 1.0 / (1.0 + exp(-x))

def dot(x, y):
    return sum(x_i * y_i for x_i, y_i in zip(x, y))

def predict(model, point):
    return logistic(dot(model, point.values))

def accuracy(data, predictions, threshold=0.5):
    correct = 0
    for d, p in zip(data, predictions):
        pred_label = 1 if p >= threshold else 0
        if pred_label == d.label:
            correct += 1
    return float(correct) / len(data)

def precision_recall_f1(data, predictions, threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for d, p in zip(data, predictions):
        pred_label = 1 if p >= threshold else 0
        true_label = d.label
        if pred_label == 1 and true_label == 1:
            tp += 1
        elif pred_label == 1 and true_label == 0:
            fp += 1
        elif pred_label == 0 and true_label == 1:
            fn += 1
        else:
            tn += 1

    precision = tp / (tp + fp) if (tp+fp) > 0 else 0
    recall = tp / (tp + fn) if (tp+fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision+recall) > 0 else 0
    return precision, recall, f1

def initialize_model(num_features):
    random.seed(1)
    return [random.gauss(0, 0.01) for _ in range(num_features)]
def train(data, epochs=10, rate=1e-3, lam=1e-3):
    num_features = len(data[0].values)
    model = initialize_model(num_features)
    N = len(data)
    for epoch in range(epochs):
        grad = [0.0]*num_features
        reg_grad = [lam * w for w in model]
        sum_loss = 0.0
        for point in data:
            p = predict(model, point)
            diff = (p - point.label)
            for i in range(num_features):
                grad[i] += diff * point.values[i]
        for i in range(num_features):
            grad[i] += lam * model[i]
        for i in range(num_features):
            model[i] -= rate * (grad[i]/N)

    return model

file_path = '/content/creditcard.csv'
data = load_data_from_csv(file_path)

# Split dataset
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

# Train
epochs = 5
learning_rate = 0.001
regularization = 0.001

model = train(train_data, epochs=epochs, rate=learning_rate, lam=regularization)

test_predictions = [predict(model, p) for p in test_data]

acc = accuracy(test_data, test_predictions, threshold=0.5)
prec, rec, f1 = precision_recall_f1(test_data, test_predictions, threshold=0.5)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)


Accuracy: 0.9811529933481153
Precision: 0.02027027027027027
Recall: 0.06818181818181818
F1 Score: 0.03125
